In [ ]:
import psycopg2
import os
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.ticker as mtick

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# PostgreSQL connection parameters

db_params = {
    
    'host': 'localhost',
    'database': 'CC_Analysis',
    'user': 'postgres',
    'password': 'root'
}

In [ ]:
def fetch_table_data(db_params):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        # Execute a query to fetch data from cc_detail table
        cur.execute("SELECT * FROM cc_detail")
        
        # Fetch all rows from the executed query
        rows = cur.fetchall()
        
        # Constructing a DataFrame
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
        
        # Display the data
        print(df)
        
        return df
    
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL table:", error)
        return None
    
    finally:
        # Close communication with the PostgreSQL database
        if conn:
            cur.close()
            conn.close()
            print("Database connection closed")

# Entry point of the script
if __name__ == '__main__':
    df = fetch_table_data(db_params)

In [ ]:
def fetch_table_data(db_params):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cur = conn.cursor()
        
        # Execute a query to fetch data from cc_detail table
        cur.execute("SELECT * FROM cust_detail")
        
        # Fetch all rows from the executed query
        rows = cur.fetchall()
        
        # Constructing a DataFrame
        df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
        
        # Display the data
        print(df)
        
        return df
    
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL table:", error)
        return None
    
    finally:
        # Close communication with the PostgreSQL database
        if conn:
            cur.close()
            conn.close()
            print("Database connection closed")

# Entry point of the script
if __name__ == '__main__':
    df2 = fetch_table_data(db_params)

In [ ]:
# Merge tables 

merged_df = pd.merge(df, df2, on = 'client_num')

In [ ]:
df['interest_earned'] = pd.to_numeric(df['interest_earned'], errors='coerce')

# Convert to integer

df['interest_earned'] = df['interest_earned'].astype(int)

In [ ]:
# Bar plot of card categories

plt.figure(figsize=(6, 3))
sns.countplot(data = merged_df, x='card_category')
plt.title('Count of Clients by Card Category')
plt.xlabel('Card Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calculate revenue

merged_df['revenue'] = merged_df['annual_fees'] + merged_df['total_trans_amt'] + merged_df['interest_earned']
merged_df['revenue'] = merged_df['revenue'].astype(int)

In [ ]:
def age_group(age):
    
    if age < 30:
        return "20-30"
    elif 30 <= age < 40:
        return "30-40"
    elif 40 <= age < 50:
        return "40-50"
    elif 50 <= age < 60:
        return "50-60"
    elif age >= 60:
        return "60+"

# Apply age_group function to create age_group column

merged_df['age_group'] = merged_df['customer_age'].apply(age_group)

In [ ]:
# Assuming df is your merged DataFrame with 'exp_type' and 'revenue' columns

revenue_by_exp_type = merged_df.groupby('exp_type')['revenue'].sum().reset_index()


# Plotting the tree map with custom figure size

fig = px.treemap(revenue_by_exp_type, path=['exp_type'], values='revenue', 
                 title='Revenue by Expense Type', hover_data=['revenue'],
                 width=600, height=400) 

fig.show()

In [ ]:
# 2. Total revenue generated by state_cd

total_revenue_by_state = merged_df.groupby('state_cd')['revenue'].sum().reset_index()


# Sort by revenue descending and select top 5 states

top_5_states = total_revenue_by_state.sort_values(by='revenue', ascending=False).head(5)




# Plotting a horizontal bar plot for top 5 states

plt.figure(figsize=(6, 3))
sns.barplot(x='revenue', y='state_cd', data=top_5_states, palette='Greens')
plt.title('Top 5 States by Total Revenue')
plt.xlabel('Revenue')
plt.ylabel('State')
plt.show()

In [ ]:
# Analyze average annual fees by age group

avg_annual_fees_by_age = merged_df.groupby('age_group')['annual_fees'].mean()


# Visualize results

plt.figure(figsize=(6,3))


# Plot average annual fees

sns.barplot(x = avg_annual_fees_by_age.index, y = avg_annual_fees_by_age.values, palette='Set2')
plt.title('Average Annual Fees by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Average Annual Fees')

plt.show()

In [ ]:
# Analyze most common card category by age group

most_common_card_by_age = merged_df.groupby('age_group')['card_category'].agg(lambda x: x.value_counts().idxmax()).reset_index()


# Plotting the most common card category

plt.figure(figsize=(5, 3))
sns.countplot(x='age_group', data = merged_df , hue='card_category', palette='Set2', order=most_common_card_by_age['age_group'])
plt.title('Most Common Card Category by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Count')

plt.legend(title='Card Category')
plt.tight_layout()
plt.show()

In [ ]:
# Select numeric columns for box plot

numeric_data = merged_df[['revenue']]

# Create box plot

plt.figure(figsize=(5, 4))
sns.boxplot(data=numeric_data)
plt.title('Boxplot of Income and Customer Age')
plt.xlabel('Variable')
plt.ylabel('Value')
plt.show()

In [ ]:
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = ((numeric_data < lower_bound) | (numeric_data > upper_bound)).any(axis=1)
clean_data = numeric_data[~outliers]

In [ ]:
plt.figure(figsize=(5, 4))
sns.boxplot(data = clean_data)

plt.title('Boxplot of Cleaned Numeric Data (IQR Method)')

plt.show()

In [ ]:
# Convert week_start_date to datetime

merged_df['week_start_date'] = pd.to_datetime(merged_df['week_start_date'])

# Set week_start_date as index

merged_df.set_index('week_start_date', inplace=True)

# Resample to monthly frequency and aggregate revenue

monthly_revenue = merged_df['revenue'].resample('M').sum()

In [ ]:
# sns.pairplot(data = df)
# plt.show()

In [ ]:
# Preprocess the data
# Selecting relevant columns

features = ['customer_age', 'gender', 'customer_job', 'income']
target = 'revenue'

In [ ]:
# Handling missing values, encoding categorical variables, and scaling numerical features

numeric_features = ['customer_age', 'income']
categorical_features = ['gender', 'customer_job']

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
# Splitting the data into training and testing sets

X = merged_df[features]
y = merged_df[target]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Creating a pipeline with the preprocessor and the Random Forest Regressor model

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])

In [ ]:
# Training the model

model.fit(X_train, y_train)

# Making predictions

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [ ]:
# Evaluating the model

train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

# Calculate RMSE as a percentage of the average revenue

average_revenue = y_test.mean()
train_rmse_percentage = (train_rmse / average_revenue) * 100
test_rmse_percentage = (test_rmse / average_revenue) * 100


print(f"Train RMSE: {train_rmse} ({train_rmse_percentage:.2f}%)")
print(f"Test RMSE: {test_rmse} ({test_rmse_percentage:.2f}%)")
print(f"Train R^2: {train_r2}")
print(f"Test R^2: {test_r2}")

In [ ]:
# Function to predict revenue for new customer data

def predict_revenue():
    
    age = int(input("Enter customer age: "))
    gender = input("Enter customer gender (Male/Female): ")
    job = input("Enter customer job: ")
    income = float(input("Enter customer income: "))
    
    input_data = pd.DataFrame({'customer_age': [age], 'gender': [gender], 'customer_job': [job], 'income': [income]})
    predicted_revenue = model.predict(input_data)[0]
    
    print(f"Predicted Revenue: {predicted_revenue}")

# Example usage

predict_revenue()